<a href="https://colab.research.google.com/github/AnasAito/Geom-viz/blob/master/dgl_to_pyviz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
## libraries 
!pip install dgl
!pip install pyvis

In [43]:
## vanilla CGN network 
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

## msg passing function 
# msg preparation 
gcn_msg = fn.copy_src(src='h', out='m')
#msg aggregation 
gcn_reduce = fn.sum(msg='m', out='h')

class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
          ## init h feature vector with feat from input of layer
            g.ndata['h'] = feature
            ## update feat vectors 
            g.update_all(gcn_msg, gcn_reduce)
            ## extract h feat vectors 
            h = g.ndata['h']
            ## projection using linear layer 
            return self.linear(h)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [9]:
## simple train and evaluate loops 
from dgl.data import citation_graph as citegrh
import networkx as nx
from dgl.data import citation_graph as citegrh
import networkx as nx
def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    train_mask = th.BoolTensor(data.train_mask)
    test_mask = th.BoolTensor(data.test_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, train_mask, test_mask

def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)



In [73]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
net = Net()
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)

dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    acc_train = evaluate(net, g, features, labels, train_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} |Train Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc,acc_train, np.mean(dur)))

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch 00000 | Loss 1.9434 | Test Acc 0.1670 |Train Acc 0.2786 | Time(s) nan


/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:285: UserWarning: Property dataset.feat will be deprecated, please use g.ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:285: UserWarning: Property dataset.label will be deprecated, please use g.ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:285: UserWarning: Property dataset.train_mask will be deprecated, please use g.ndata['train_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:285: UserWarning: Property dataset.test_mask will be deprecated, please use g.ndata['test_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


Epoch 00001 | Loss 1.8059 | Test Acc 0.2190 |Train Acc 0.3714 | Time(s) nan
Epoch 00002 | Loss 1.6806 | Test Acc 0.3280 |Train Acc 0.5071 | Time(s) nan
Epoch 00003 | Loss 1.5690 | Test Acc 0.4430 |Train Acc 0.6071 | Time(s) 0.0398
Epoch 00004 | Loss 1.4701 | Test Acc 0.5250 |Train Acc 0.6786 | Time(s) 0.0407
Epoch 00005 | Loss 1.3824 | Test Acc 0.5720 |Train Acc 0.7214 | Time(s) 0.0394
Epoch 00006 | Loss 1.3044 | Test Acc 0.5930 |Train Acc 0.7429 | Time(s) 0.0407
Epoch 00007 | Loss 1.2337 | Test Acc 0.5940 |Train Acc 0.7429 | Time(s) 0.0399
Epoch 00008 | Loss 1.1650 | Test Acc 0.5980 |Train Acc 0.7786 | Time(s) 0.0393
Epoch 00009 | Loss 1.0977 | Test Acc 0.6140 |Train Acc 0.8143 | Time(s) 0.0395
Epoch 00010 | Loss 1.0296 | Test Acc 0.6350 |Train Acc 0.8286 | Time(s) 0.0394
Epoch 00011 | Loss 0.9606 | Test Acc 0.6490 |Train Acc 0.8357 | Time(s) 0.0397
Epoch 00012 | Loss 0.8936 | Test Acc 0.6500 |Train Acc 0.8429 | Time(s) 0.0397
Epoch 00013 | Loss 0.8307 | Test Acc 0.6570 |Train Acc 0.8

In [96]:
def has_dst_one(edges):
  print(edges.edges())

g.filter_edges(has_dst_one)

(tensor([   0,    0,    0,  ..., 2707, 2707, 2707]), tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]), tensor([    0,     1,     2,  ..., 10553, 10554, 10555]))


AttributeError: ignored

In [100]:
np.array([True,False]).squeeze(1)

AxisError: ignored

In [106]:

from pyvis.network import Network
import networkx as nx


g_render = Network('500px', '800px',notebook=True)

# filter edge_data 
def get_edges(g,labels , mask , with_mask = False):
     src,dst = g.all_edges()
     src,dst = src.detach().numpy(),dst.detach().numpy()
     group_list = labels.detach().numpy()
  
     edge_data = zip(src,dst)

     if with_mask : 
       src =[]
       dst =[]
       filtred_nodes = g.nodes()[mask].detach().numpy()
       for edge in edge_data : 
         #print(edge)
         node_a,node_b = edge
         if (node_a in filtred_nodes) or (node_b in filtred_nodes):
           src.append(node_a)
           dst.append(node_b)
      
       edge_data = zip(np.array(src),np.array(dst))
 
     return edge_data,group_list

edge_data,group_list = get_edges(g,labels , mask = train_mask , with_mask = True)
# populates the nodes and edges data structures
for e in edge_data:
    src = str(e[0])
    dst = str(e[1])
    label_src = str(group_list[e[0]])
    label_dst = str(group_list[e[1]])
   # print(label_src)
    g_render.add_node(src,  title=src, group=label_src)
    g_render.add_node(dst,  title=dst, group=label_dst)
    g_render.add_edge(src, dst,)

g_render.show_buttons(filter_=['physics'])
g_render.show('nx.html')
import IPython
IPython.display.HTML(filename='/content/nx.html')

In [77]:
g_render.show_buttons(filter_=['physics'])
g_render.show('nt.html')
#import IPython
#IPython.display.HTML(filename='/content/nx.html')

TypeError: ignored

In [40]:
g.ndata

{'h': tensor([[0.0453, 0.1327, 0.0000,  ..., 0.8593, 2.1278, 1.5715],
        [0.7761, 1.9361, 0.0000,  ..., 0.1356, 0.0822, 0.2471],
        [1.8366, 7.8984, 0.0000,  ..., 1.1115, 2.8987, 0.8423],
        ...,
        [0.1754, 0.2267, 0.0000,  ..., 0.2568, 0.2396, 0.3689],
        [0.3103, 1.7619, 0.0000,  ..., 0.6582, 1.1252, 0.9099],
        [0.6374, 1.8340, 0.0000,  ..., 2.7867, 4.7241, 3.6294]])}